In [1]:
library(dplyr)
library(Hmisc)
library(igraph)

Warning message:
“package ‘dplyr’ was built under R version 4.2.3”

Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Warning message:
“package ‘Hmisc’ was built under R version 4.2.3”

Attaching package: ‘Hmisc’


The following objects are masked from ‘package:dplyr’:

    src, summarize


The following objects are masked from ‘package:base’:

    format.pval, units




ERROR: Error: package or namespace load failed for ‘igraph’ in dyn.load(file, DLLpath = DLLpath, ...):
 unable to load shared object '/root/miniconda3/envs/Rdoc/lib/R/library/igraph/libs/igraph.so':
  libglpk.so.36: cannot open shared object file: No such file or directory


In [3]:
species.df = read.csv('./4.cor/rename.species.expr.csv',row.names = 1)

In [5]:
species.df_t = species.df

In [6]:
trt_id <- c('KO.CLP','KO.NC','WT.CLP','WT.NC') ## 定义样本的关键词，然后从样本名抓取处理的样本
split_otu <- lapply(apply(sapply(trt_id,function(x){grep(x,colnames(species.df_t))}),2,FUN = function(x){species.df_t[,x]}),function(x){x[-(which(rowSums(x)==0)),]})

In [7]:
split_otu1 = split_otu

In [8]:
library(readxl)

Warning message:
“package ‘readxl’ was built under R version 4.2.3”


In [9]:
library(openxlsx)

Warning message:
“package ‘openxlsx’ was built under R version 4.2.3”


In [10]:
data <- read.xlsx("../daixiezu/新非靶/3.Data/data.xlsx", sheet = 2,check.names = T)

In [11]:
data = data[-1,-1]

In [12]:
library(tibble)
data = as.tibble(data)
data = data[!duplicated(data$Metabolites),]

Warning message:
“package ‘tibble’ was built under R version 4.2.3”
Warning message:
“`as.tibble()` was deprecated in tibble 2.0.0.
ℹ Please use `as_tibble()` instead.
ℹ The signature and semantics have changed, see `?as_tibble`.”


In [13]:
data %>% tibble::column_to_rownames(.,'Metabolites') -> data1

In [14]:
dim(data1)

[1] 144  32

In [25]:
data_t = as.data.frame(lapply(data1,as.numeric))

In [26]:
rownames(data_t) = rownames(data1)

In [27]:
data_t = data_t['Adrenic Acid',]

In [28]:
head(data_t,2)

,KO.CLP1,KO.CLP2,KO.CLP3,KO.CLP4,KO.CLP5,KO.CLP6,KO.CLP7,KO.CLP8,KO.NC1,KO.NC2,⋯,WT.CLP7,WT.CLP8,WT.NC1,WT.NC2,WT.NC3,WT.NC4,WT.NC5,WT.NC6,WT.NC7,WT.NC8
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Adrenic Acid,21492.96,37074.22,31136.33,26490.2,42590.78,18898.78,27984.36,21294.64,9716.224,20680.1,⋯,37545.29,31127.43,0,11750.62,7426.261,0,23061.63,0,14938.64,0


In [29]:
trt_id <- c('KO.CLP','KO.NC','WT.CLP','WT.NC') ## 定义样本的关键词，然后从样本名抓取处理的样本

In [30]:
split_metainfo = list()
for (i in trt_id){
    split_metainfo[[i]] = dplyr::select(data_t,contains(i))
}

In [31]:
merge_cor = list()
otu = list()
meta = list()
p = list()
r = list()

for (i in c(1,2,3,4)){
    otu[[i]] = split_otu1[[i]]
    meta[[i]] = split_metainfo[[i]]
    library(psych)
    merge_cor[[i]]<- corr.test(as.data.frame(t(otu[[i]])), as.data.frame(t(meta[[i]])), method = 'spearman')
    p[[i]] = merge_cor[[i]]$p
    r[[i]] = merge_cor[[i]]$r
    r[[i]][p[[i]]>0.05|abs(r[[i]])<0.2] = 0
    r[[i]][is.na(r[[i]])]=0
    p[[i]][is.na(p[[i]])]=0
    outdir = paste0(getwd(),'/6.otu-metabolite-luming-adrenic')
    if(dir.exists(outdir)){
      print("dir exists")
    }else{
      dir.create(outdir)
    }
    write.csv(as.data.frame(t(r[[i]])),file = paste0(outdir,'/',trt_id[i],'.cor.r.csv'))
    write.csv(as.data.frame(t(p[[i]])),file = paste0(outdir,'/',trt_id[i],'.cor.p.csv'))
}

Warning message:
“package ‘psych’ was built under R version 4.2.3”

Attaching package: ‘psych’


The following object is masked from ‘package:Hmisc’:

    describe




[1] "dir exists"
[1] "dir exists"
[1] "dir exists"
